# 1. Importing Required Libraries

In [201]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# 2. Reading the Data

In [202]:
PROJECT_DIR = r"D:\Flights-sagemaker-project"
DATA_DIR = "data"

In [203]:
def get_data(name):
    file_name = f"{name}.csv"
    file_path = os.path.join(PROJECT_DIR, DATA_DIR,file_name)
    return pd.read_csv(file_path)

In [204]:
flights = get_data("flight_price")
flights.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [205]:
flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


* The dataset contains 10683 rows and 11 features 
* Columns `Route` and ` Total_Stops` have missing value each
* The data types of some features isn't appropriate

# 3. Preliminary Analysis


## 3.1 Check Data Types

In [206]:
flights.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [207]:
flights.dtypes

Airline            object
Date_of_Journey    object
Source             object
Destination        object
Route              object
Dep_Time           object
Arrival_Time       object
Duration           object
Total_Stops        object
Additional_Info    object
Price               int64
dtype: object

In [208]:
flights['Date_of_Journey'][3]

'12/05/2019'

In [209]:
type(flights['Date_of_Journey'][3])

str

In [210]:
flights['Dep_Time'][3]

'18:05'

In [211]:
type(flights['Dep_Time'][3])

str

## 3.2 Check for Duplicated

In [212]:
(
    flights.loc[flights.duplicated(keep = False)]
    .sort_values(["Airline", "Date_of_Journey","Source"])

)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
6321,Air India,01/03/2019,Banglore,New Delhi,BLR → BOM → AMD → DEL,08:50,23:55 02 Mar,39h 5m,2 stops,No info,17135
9848,Air India,01/03/2019,Banglore,New Delhi,BLR → BOM → AMD → DEL,08:50,23:55 02 Mar,39h 5m,2 stops,No info,17135
572,Air India,03/03/2019,Banglore,New Delhi,BLR → DEL,21:10,23:55,2h 45m,non-stop,No info,7591
8168,Air India,03/03/2019,Banglore,New Delhi,BLR → DEL,21:10,23:55,2h 45m,non-stop,No info,7591
1495,Air India,1/04/2019,Kolkata,Banglore,CCU → DEL → COK → BLR,10:00,01:20 02 Apr,15h 20m,2 stops,No info,10408
...,...,...,...,...,...,...,...,...,...,...,...
2692,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,05:45,08:35,2h 50m,non-stop,No check-in baggage included,4273
2870,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,05:45,08:35,2h 50m,non-stop,No check-in baggage included,4273
3711,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,20:30,23:20,2h 50m,non-stop,No check-in baggage included,3873
2634,Vistara,24/03/2019,Banglore,New Delhi,BLR → DEL,11:30,14:10,2h 40m,non-stop,No info,5403


## 3.3 Observations

* The type of `Date_of Journey`, `Dep_Time` and `Arrival Time` should be changed to datatime 
* The type of `Duration`, `total_stop` is mixed they shoud be numeric type
* There are 220 Duplicates. These should be removed.
  

# 4. Detailed Analysis

### Airline

In [213]:
flights.Airline

0             IndiGo
1          Air India
2        Jet Airways
3             IndiGo
4             IndiGo
            ...     
10678       Air Asia
10679      Air India
10680    Jet Airways
10681        Vistara
10682      Air India
Name: Airline, Length: 10683, dtype: object

In [214]:
flights.Airline.unique()

array(['IndiGo', 'Air India', 'Jet Airways', 'SpiceJet',
       'Multiple carriers', 'GoAir', 'Vistara', 'Air Asia',
       'Vistara Premium economy', 'Jet Airways Business',
       'Multiple carriers Premium economy', 'Trujet'], dtype=object)

* Some airline are same but has differnt name

In [215]:
(
    flights
    .Airline
    .str.replace("Vistara Premium economy","Vistara")
    .str.replace("Multiple carriers Premium economy", "Multiple carriers")
    .str.replace("Jet Airways Business","Jet Airways")
#     .unique()
    .str.title()
)

0             Indigo
1          Air India
2        Jet Airways
3             Indigo
4             Indigo
            ...     
10678       Air Asia
10679      Air India
10680    Jet Airways
10681        Vistara
10682      Air India
Name: Airline, Length: 10683, dtype: object

### Date_of_journey

In [216]:
flights.Date_of_Journey

0        24/03/2019
1         1/05/2019
2         9/06/2019
3        12/05/2019
4        01/03/2019
            ...    
10678     9/04/2019
10679    27/04/2019
10680    27/04/2019
10681    01/03/2019
10682     9/05/2019
Name: Date_of_Journey, Length: 10683, dtype: object

In [217]:
pd.to_datetime(flights.Date_of_Journey, dayfirst= True)

0       2019-03-24
1       2019-05-01
2       2019-06-09
3       2019-05-12
4       2019-03-01
           ...    
10678   2019-04-09
10679   2019-04-27
10680   2019-04-27
10681   2019-03-01
10682   2019-05-09
Name: Date_of_Journey, Length: 10683, dtype: datetime64[ns]

### Source

In [218]:
flights.Source.unique()

array(['Banglore', 'Kolkata', 'Delhi', 'Chennai', 'Mumbai'], dtype=object)

### Destination

In [219]:
flights.Destination.unique()

array(['New Delhi', 'Banglore', 'Cochin', 'Kolkata', 'Delhi', 'Hyderabad'],
      dtype=object)

### Dep_time

In [220]:
flights.Dep_Time

0        22:20
1        05:50
2        09:25
3        18:05
4        16:50
         ...  
10678    19:55
10679    20:45
10680    08:20
10681    11:30
10682    10:55
Name: Dep_Time, Length: 10683, dtype: object

In [221]:
(
    flights
    .Dep_Time
    .loc[lambda ser: ser.str.contains("[^0-9:]")]

)

Series([], Name: Dep_Time, dtype: object)

In [222]:
pd.to_datetime(flights.Dep_Time).dt.time

0        22:20:00
1        05:50:00
2        09:25:00
3        18:05:00
4        16:50:00
           ...   
10678    19:55:00
10679    20:45:00
10680    08:20:00
10681    11:30:00
10682    10:55:00
Name: Dep_Time, Length: 10683, dtype: object

### Arrival Time

In [223]:
(
    flights
    .Arrival_Time
    .loc[lambda sec : sec.str.contains("[^0-9:]")]
    .str.split(" ", n= 1)
    .str.get(-1)
    .unique()
)

array(['22 Mar', '10 Jun', '13 Mar', '02 Mar', '10 May', '04 Mar',
       '13 Jun', '28 May', '19 Mar', '07 May', '02 Jun', '16 Jun',
       '19 May', '16 May', '28 Jun', '02 May', '28 Mar', '19 Jun',
       '04 Apr', '25 Mar', '07 Mar', '25 Jun', '07 Jun', '25 May',
       '13 May', '16 Mar', '22 May', '10 Apr', '04 Jun', '20 May',
       '28 Apr', '25 Apr', '10 Mar', '19 Apr', '13 Apr', '02 Apr',
       '23 Mar', '22 Apr', '11 May', '07 Apr', '03 May', '08 Mar',
       '03 Mar', '05 Mar', '22 Jun', '04 May', '26 May', '16 Apr',
       '26 Jun', '29 May', '29 Jun', '29 Mar', '23 May', '17 Jun'],
      dtype=object)

### Duration    

In [224]:
(
    flights
    .Duration
    .loc[lambda ser: ~ser.str.contains("m")]
    
)

2        19h
18       23h
33       22h
44       12h
53        3h
        ... 
10591    23h
10638    14h
10639    38h
10673    15h
10680     3h
Name: Duration, Length: 1031, dtype: object

In [225]:
(
    flights
    .Duration
    .loc[lambda ser: ~ser.str.contains("h")]
    
)

6474    5m
Name: Duration, dtype: object

In [226]:
flights.iloc[[6474]]

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
6474,Air India,6/03/2019,Mumbai,Hyderabad,BOM → GOI → PNQ → HYD,16:50,16:55,5m,2 stops,No info,17327


* The observation 6474 has incorrect value

In [227]:
(
    flights
    .Duration
    .drop(index = 6474)
    .str.split(" ", expand = True)  # expand convets into dataframe
    .set_axis(["hour","minute"], axis= 1)
    .assign(
            hour = lambda df_:(
            df_
            .hour
            .str.replace("h","")
            .astype(int)
            .mul(60)
            
    ),
    minute = lambda df_:(
             df_
            .minute
            .str.replace("m","")
            .fillna(0)
             .astype(int)
    )
    )
    .sum(axis = 1)
)

0         170
1         445
2        1140
3         325
4         285
         ... 
10678     150
10679     155
10680     180
10681     160
10682     500
Length: 10682, dtype: int64

### Total Stops


In [228]:
flights.Total_Stops.unique()

array(['non-stop', '2 stops', '1 stop', '3 stops', nan, '4 stops'],
      dtype=object)

In [253]:
flights[flights.Total_Stops.isna()]

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
9039,Air India,6/05/2019,Delhi,Cochin,NaN,09:45,09:25 07 May,23h 40m,NaN,No info,7480


In [254]:
(
    flights
    .Total_Stops
    .replace("non-stop", "0")
    .str.replace("stops?","", regex= True)  #? will consider "stop" or "stops"
    .pipe(lambda ser: pd.to_numeric(ser))
)

0        0.0
1        2.0
2        2.0
3        1.0
4        1.0
        ... 
10678    0.0
10679    0.0
10680    0.0
10681    0.0
10682    2.0
Name: Total_Stops, Length: 10683, dtype: float64

### Additional info

In [255]:
flights.Additional_Info.unique()

array(['No info', 'In-flight meal not included',
       'No check-in baggage included', '1 Short layover', 'No Info',
       '1 Long layover', 'Change airports', 'Business class',
       'Red-eye flight', '2 Long layover'], dtype=object)

# 5. Cleaning Operations

In [256]:
def convert_to_minutes(ser):
    return (
            ser
            .str.split(" ", expand = True)  # expand convets into dataframe
            .set_axis(["hour","minute"], axis= 1)
            .assign(
                    hour = lambda df_:(
                    df_
                    .hour
                    .str.replace("h","")
                    .astype(int)
                    .mul(60)

            ),
            minute = lambda df_:(
                     df_
                    .minute
                    .str.replace("m","")
                    .fillna(0)
                     .astype(int)
                )
            )
            .sum(axis = 1)
        )
    

In [250]:
def clean_data(df):
    
    return (
        df
        .drop(index = 6474)
#         .drop(index = 9039)
        .drop_duplicates()
        .assign(**{
            col: df[col].str.strip()
            for col in df.select_dtypes(include="O").columns
        })
        .rename(columns= str.lower)
        .assign(
            airline = lambda df_: (
            df_
                .airline
                .str.replace("Vistara Premium economy","Vistara")
                .str.replace("Multiple carriers Premium economy", "Multiple carriers")
                .str.replace("Jet Airways Business","Jet Airways")
                .str.title()
            ),
            date_of_journey = lambda df_: pd.to_datetime(df_.date_of_journey, dayfirst = True),
            dep_time = lambda df_: pd.to_datetime(df_.dep_time).dt.time,
            arrival_time = lambda df_: pd.to_datetime(df_.arrival_time).dt.time,
            duration = lambda df_: df_.duration.pipe(convert_to_minutes),
            total_stops = lambda df_: (
                            df_
                            .total_stops
                            .replace("non-stop", "0")
                            .str.replace("stops?","", regex= True)  #? will consider "stop" or "stops"
                            .pipe(lambda ser: pd.to_numeric(ser))
                        ),
            additional_info = lambda df_: df_.additional_info.replace("No info","No Info")
        )
        .drop(columns="route")
        
    )

In [251]:
flights_cleaned = clean_data(flights)

In [252]:
flights_cleaned

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Indigo,2019-03-24,Banglore,New Delhi,22:20:00,01:10:00,170,0.0,No Info,3897
1,Air India,2019-05-01,Kolkata,Banglore,05:50:00,13:15:00,445,2.0,No Info,7662
2,Jet Airways,2019-06-09,Delhi,Cochin,09:25:00,04:25:00,1140,2.0,No Info,13882
3,Indigo,2019-05-12,Kolkata,Banglore,18:05:00,23:30:00,325,1.0,No Info,6218
4,Indigo,2019-03-01,Banglore,New Delhi,16:50:00,21:35:00,285,1.0,No Info,13302
...,...,...,...,...,...,...,...,...,...,...
10678,Air Asia,2019-04-09,Kolkata,Banglore,19:55:00,22:25:00,150,0.0,No Info,4107
10679,Air India,2019-04-27,Kolkata,Banglore,20:45:00,23:20:00,155,0.0,No Info,4145
10680,Jet Airways,2019-04-27,Banglore,Delhi,08:20:00,11:20:00,180,0.0,No Info,7229
10681,Vistara,2019-03-01,Banglore,New Delhi,11:30:00,14:10:00,160,0.0,No Info,12648


# 6. Split the Data

In [238]:
flights_final = flights_cleaned

In [239]:
X = flights_final.drop(columns="price")
y = flights_final.price.copy()

In [240]:
X_train, X_, y_train, y_ = train_test_split(X, y, test_size=0.3,random_state = 123)
X_test, X_val, y_test, y_val = train_test_split(X_, y_, test_size=0.5,random_state = 123)

In [241]:
X_train.shape

(7322, 9)

In [242]:
X_test.shape

(1569, 9)

In [243]:
X_val.shape

(1570, 9)

# 7. Export the subset

In [244]:
def export_data(X,y,name):
    file_name = f"{name}.csv"
    file_path = os.path.join(PROJECT_DIR, DATA_DIR, file_name)
    
    X.join(y).to_csv(file_path, index = False)
    
#     return pd.read_csv(file_path)
    

In [245]:
export_data(X_train,y_train, "train")
export_data(X_test,y_test, "test")
export_data(X_val,y_val, "val")